In [ ]:
import numpy as np
import pandas as pd
import csv
import sys
import hashlib
import re
import string
import itertools
# use natural language toolkit

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

lemma = nltk.wordnet.WordNetLemmatizer()
sno = nltk.stem.SnowballStemmer('english')

In [ ]:
DATA_LOC = 'C:\\Users\\pvaish10\\Desktop\\'  # !! Modify this to the CSV data location

In [ ]:
train = pd.read_csv(filepath_or_buffer = DATA_LOC)

In [ ]:
train.head(10)

##### Extracting Features

In [ ]:
VerbCombos = ['VB',
              'VBD',
              'VBG',
              'VBN',
              'VBP',
              'VBZ',
              'WDT',
              'WP',
              'WP$',
              'WRB',
              'MD']

questionTriples = ['CD-VB-VBN',
                   'MD-PRP-VB' ,
                   'MD-VB-CD' ,
                   'NN-IN-DT' ,
                   'PRP-VB-PRP' ,
                   'PRP-WP-NNP' ,
                   'VB-CD-VB' ,
                   'VB-PRP-WP' ,
                   'VBZ-DT-NN' ,
                   'WP-VBZ-DT' ,
                   'WP-VBZ-NNP' ,
                   'WRB-MD-VB']

statementTriples = ['DT-JJ-NN',
                   'DT-NN-VBZ',
                   'DT-NNP-NNP',
                   'IN-DT-NN',
                   'IN-NN-NNS',
                   'MD-VB-VBN',
                   'NNP-IN-NNP',
                   'NNP-NNP-NNP',
                   'NNP-VBZ-DT',
                   'NNP-VBZ-NNP',
                   'NNS-IN-DT',
                   'VB-VBN-IN',
                   'VBZ-DT-JJ']


startTuples = ['NNS-DT',
               'WP-VBZ',
               'WRB-MD']                      

endTuples = ['IN-NN',
             'VB-VBN',
             'VBZ-NNP'] 

In [ ]:
# use natural language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
# word stemmer
stemmer = LancasterStemmer()
import string

In [ ]:
def strip_sentence(sentence):
    sentence = sentence.strip(",")
    sentence = ''.join(filter(lambda x: x in string.printable, sentence))  #strip out non-alpha-numerix
    sentence = sentence.translate(str.maketrans('','',string.punctuation)) #strip punctuation
    return(sentence)

# Pass in a list of strings (i.e. PoS types) and the sentence to check PoS types for
# check if *Any Pair Combo* of the PoS types list exists in the sentence PoS types
# return a count of occurrence
def exists_pair_combos(comboCheckList, sentence):
    pos = get_pos(sentence)
    tag_string = "-".join([ i[1] for i in pos ])
    combo_list = []
    
    for pair in itertools.permutations(comboCheckList,2):
        if(pair[0] == "MD"):  # * Kludge - strip off leading MD *
            pair = ["",""]
        combo_list.append("-".join(pair))
    
    if any(code in tag_string for code in combo_list):
	    return 1
    else:
        return 0
    
# Parts Of Speech
def get_pos(sentence):
    sentenceParsed = nltk.word_tokenize(sentence)
    return(nltk.pos_tag(sentenceParsed))
    
# Count Q-Marks    
def count_qmark(sentence):
    return(sentence.count("?") )
    
# Count a specific POS-Type
#VBG = count_POSType(pos,'VBG')
def count_POSType(pos, ptype):
    count = 0
    tags = [ i[1] for i in pos ]
    return(tags.count(ptype))
    #if ptype in tags:
    #    VBG = 1
    #return(VBG)
    
# Does Verb occur before first Noun
def exists_vb_before_nn(pos):
    pos_tags = [ i[1] for i in pos ]
    #Strip the Verbs to all just "V"
    pos_tags = [ re.sub(r'V.*','V', str) for str in pos_tags ]
    #Strip the Nouns to all just "NN"
    pos_tags = [ re.sub(r'NN.*','NN', str) for str in pos_tags ]
    
    vi =99
    ni =99
    mi =99
    
    #Get first NN index
    if "NN" in pos_tags:
        ni = pos_tags.index("NN")
    #Get first V index
    if "V" in pos_tags:
        vi = pos_tags.index("V")
    #get Modal Index
    if "MD" in pos_tags:
        mi = pos_tags.index("MD")

    if vi < ni or mi < ni :
        return(1)
    else:
        return(0)

# Stemmed sentence ends in "NN-NN"?  
def exists_stemmed_end_NN(stemmed):
    stemmedEndNN = 0
    stemmed_end = get_first_last_tuples(" ".join(stemmed))[1]
    if stemmed_end == "NN-NN":
        stemmedEndNN = 1
    return(stemmedEndNN)

# Go through the predefined list of start-tuples, 1 / 0 if given startTuple occurs in the list
def exists_startTuple(startTuple):    
    exists_startTuples = []
    for tstring in startTuples:  #startTuples defined as global var
        if startTuple in tstring:
            exists_startTuples.append(1)
        else:
            exists_startTuples.append(0)
        return(exists_startTuples)

# Go through the predefined list of end-tuples, 1 / 0 if given Tuple occurs in the list    
def exists_endTuple(endTuple):
    exists_endTuples = []
    for tstring in endTuples:    #endTuples defined as global var
        if endTuple in tstring:
            exists_endTuples.append(1)
        else:
            exists_endTuples.append(0)
    return(exists_endTuples)

#loop round list of triples and construct a list of binary 1/0 vals if triples occur in list
def exists_triples(triples, tripleSet):
    exists = []
    for tstring in tripleSet:   
        if tstring in triples:
            exists.append(1)
        else:
            exists.append(0)
    return(exists)

# Get a sentence and spit out the POS triples
def get_triples(pos):
    list_of_triple_strings = []
    pos = [ i[1] for i in pos ]  # extract the 2nd element of the POS tuples in list
    n = len(pos)
    
    if n > 2:  # need to have three items
        for i in range(0,n-2):
            t = "-".join(pos[i:i+3]) # pull out 3 list item from counter, convert to string
            list_of_triple_strings.append(t)
    return list_of_triple_strings 
        
def get_first_last_tuples(sentence):
    first_last_tuples = []
    sentenceParsed = nltk.word_tokenize(sentence)
    pos = nltk.pos_tag(sentenceParsed) #Parts Of Speech
    pos = [ i[1] for i in pos ]  # extract the 2nd element of the POS tuples in list
    
    n = len(pos)
    first = ""
    last = ""
    
    if n > 1:  # need to have three items
        first = "-".join(pos[0:2]) # pull out first 2 list items
        last = "-".join(pos[-2:]) # pull out last 2 list items
    
    first_last_tuples = [first, last]
    return first_last_tuples

def lemmatize(sentence):    
    """
    pass  in  a sentence as a string, return just core text that has been "lematised"
    stop words are removed - could effect ability to detect if this is a question or answer
    - depends on import lemma = nltk.wordnet.WordNetLemmatizer() and from nltk.corpus import stopwords
    """
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence)
    
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w.lower())  # also set lowercase
    lem = []        
    for w in filtered_sentence:
        lem.append(lemma.lemmatize(w))
  
    return lem    

def stematize(sentence):
    """
    pass  in  a sentence as a string, return just core text stemmed
    stop words are removed - could effect ability to detect if this is a question or answer
    - depends on import sno = nltk.stem.SnowballStemmer('english') and from nltk.corpus import stopwords
    """
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(sentence)
    
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    stemmed = []        
    for w in filtered_sentence:
        stemmed.append(sno.stem(w))
  
    return stemmed    


###### Count number of question marks?

In [ ]:
train['qMark'] = train.apply(lambda row: count_qmark(row['SENTENCE']), axis=1)

#### length of the sentence

In [ ]:
train['wordCount'] = train.apply(lambda row: len(row['SENTENCE'].split()), axis=1)

In [ ]:
stop = stopwords.words('english')
train['text_without_stopwords'] = (train['SENTENCE'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop) ])))

In [ ]:
#train['word_tokens'] = train.apply(lambda row: nltk.word_tokenize(row['SENTENCE']), axis=1)

In [ ]:
#train['stemmed']  =  train.apply(lambda row: stematize(row['SENTENCE']), axis=1)

In [ ]:
train['lemma'] = train.apply(lambda row: lemmatize(row['SENTENCE']), axis=1)

In [ ]:
#train['stemmedCount'] = train.apply(lambda row: len(row['stemmed']), axis=1)

In [ ]:
train['lemmaCount'] = train.apply(lambda row: len(row['lemma']), axis=1)

# Lemma or Stemm???

In [ ]:
train['qVerbCombo'] = train.apply(lambda row: exists_pair_combos(VerbCombos,row['SENTENCE']), axis=1)

In [ ]:
#train["stemmedEndNN"] = train.apply(lambda row: exists_stemmed_end_NN(row['stemmed']), axis=1)

In [ ]:
train["lemmaEndNN"] = train.apply(lambda row: exists_stemmed_end_NN(row['lemma']), axis=1)

In [ ]:
train['pos'] = train.apply(lambda row: get_pos(row['SENTENCE']), axis=1)

In [ ]:
for ptype in ["VBG", "VBZ", "NNP", "NN", "NNS", "NNPS","PRP", "CD" ]:
        train['_' + str(ptype)] = train['pos'].apply(lambda row: count_POSType(row,ptype))

In [ ]:
train['verbBeforeNoun'] = train.apply(lambda row: exists_vb_before_nn(row['pos']), axis=1)

In [ ]:
train['get_triples'] = train.apply(lambda row: get_triples(row['pos']), axis=1)

In [ ]:
train.info()

In [ ]:
train.columns

###### Rearrange the columns

In [ ]:
train = train[['SENTENCE', 'qMark', 'wordCount', 'text_without_stopwords',
       'lemma', 'lemmaCount', 'qVerbCombo', 'lemmaEndNN', 'pos', '_VBG',
       '_VBZ', '_NNP', '_NN', '_NNS', '_NNPS', '_PRP', '_CD', 'verbBeforeNoun',
       'get_triples', 'CLASS']]

In [ ]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()

In [ ]:
train['text_without_stopwords'] = class_le.fit_transform(train['text_without_stopwords'].values)
train['SENTENCE'] = class_le.fit_transform(train['SENTENCE'].values)
train['lemma'] = class_le.fit_transform(train['lemma'].values)
train['get_triples'] = class_le.fit_transform(train['get_triples'].values)
train['CLASS'] = class_le.fit_transform(train['CLASS'].values)

In [ ]:
train

##### Split the dataset into train and test

In [ ]:
msk = np.random.rand(len(train)) < 0.8
train_1 = train[msk]
test = train[~msk]

In [ ]:
print(len(train_1))
print(len(test))

#### Select the test dataset without the CLASS

In [ ]:
test_1 = test[test.columns.difference(['CLASS'])]

##### Join the two datasets and add a target

In [ ]:
combine = pd.concat([train_1.drop('CLASS',1),test_1])
response = train_1['CLASS']

In [ ]:
print(combine.shape)
print(response.shape)

#### Feature Selection

In [ ]:
train_subset = combine[['SENTENCE','qMark', 'wordCount', 'text_without_stopwords', 'lemma', 'lemmaCount', 'qVerbCombo',
       'lemmaEndNN', '_VBG', '_VBZ', '_NNP', '_NN', '_NNS', '_NNPS',
       '_PRP', '_CD', 'verbBeforeNoun', 'get_triples']]

In [ ]:
train_subset.info()

#### Separate the target variable and the training and test data set

In [ ]:
X = train_subset[:train_1.shape[0]]
test_2 = train_subset[train_1.shape[0]:]

In [ ]:
print (str(len(train_1))+" rows for training set")
print (str(len(test_2))+" rows for test set")
print (str(len(response))+" rows for target")

In [ ]:
checkResponse = test['CLASS']
print(len(checkResponse))

In [ ]:
test

In [ ]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X, response)
test_y_1 = logreg.predict(test_2)
#Comparing the test column with labels against the predictions
accuracy_score(checkResponse, test_y_1)

##### Confusion Matrix for Logistic regression

In [ ]:
target_names = ['No', 'Yes']
print(classification_report(checkResponse, test_y_1, target_names=target_names))

##### Transform back to labels

In [ ]:
test_y_1

In [ ]:
test_y_Transform = class_le.inverse_transform(test_y_1)

In [ ]:
test_y_Transform